In [108]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os.path
from PIL import Image
from imblearn.under_sampling import RandomUnderSampler
import random
import math

In [109]:
from sklearn.model_selection import GroupShuffleSplit, GroupKFold
from sklearn.metrics import auc, roc_curve, confusion_matrix, average_precision_score, precision_recall_curve
from sklearn.utils.class_weight import compute_class_weight

In [110]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, Sampler
from torch.optim import AdamW, Adam
from torchvision.transforms import transforms, v2
from torchvision.models import vgg16
import torchvision.models as models

# settings 

In [111]:
device = ("cuda" if torch.cuda.is_available() 
          else "mps" if torch.backends.mps.is_available() 
          else "cpu")
print(f"device: {device}")

device: mps


In [112]:
torch.manual_seed(1729)
device_gen = torch.Generator(device)
torch.set_default_device(device)

In [113]:
IN_KAGGLE = "KAGGLE_KERNEL_RUN_TYPE" in os.environ
IN_COLAB = "COLAB_GPU" in os.environ

if IN_COLAB:
    from google.colab import userdata    
    # only the first time
    # os.environ["KAGGLE_KEY"] = "d0ebb5786a5a5439881827924cf0ccbd"
    # os.environ["KAGGLE_USERNAME"] = "yuda03979"
    # 
    # ! kaggle competitions download isic-2024-challenge
    # ! unzip isic-2024-challenge.zip
    IMG_DIR = "/content/train-image/image"
    CSV_PATH = "/content/train-metadata.csv"
elif IN_KAGGLE:
    IMG_DIR = "/kaggle/input/isic-2024-challenge/train-image/image"
    CSV_PATH = "/kaggle/input/isic-2024-challenge/train-metadata.csv"
    IRRELEVANT_IMGS_PATH = "/kaggle/input/irrelevant-images/top_10000_img_names.csv" # get from benzi
else:
    IMG_DIR = "/Users/yuda/Desktop/data_bases/isic-2024-challenge/train-image/image"
    CSV_PATH = "/Users/yuda/PycharmProjects/my_isic2024/isic-2024-challenge/train-metadata.csv"

In [114]:
torch.manual_seed(1729)

In [115]:
def roc_auc_80(y_pred, y):
    fpr, tpr, thresholds = roc_curve(y, y_pred)
    tpr_80 = [0.80 if i >= 0.80 else i for i in tpr]
    return auc(fpr, tpr) - auc(fpr, tpr_80)

def pr_rc_auc(y_pred, y):
    precision, recall, thresholds = precision_recall_curve(y, y_pred)
    return auc(recall, precision)

# data and constance

In [116]:
BATCH_SIZE = 32
NUM_EPOCHS = 50
IMG_SIZE = (150, 150)
NUM_CLASSES = 2
LR = 3e-5
RANDOM_STATE = 42
CLASS_1_AMOUNT_PER_BATCH = BATCH_SIZE // 2
TRAIN_CLASS_1_SIZE, VAL_CLASS_1_SIZE, TRAIN_CLASS_0_SIZE, VAL_CLASS_0_SIZE = None, None, 5000, 5000
N_SPLITS = 10
NAMES_SAVED_MODELS = f"04_09_2024_{IMG_SIZE[0]}_{NUM_CLASSES}"
T_0 = NUM_EPOCHS * ((TRAIN_CLASS_0_SIZE + 300) // BATCH_SIZE)
MORE_IMGS = False

In [ ]:
MODELS_NAMES = [
#     "mobilenet_v3_small",
#     "efficientnet_v2_m",
#     "efficientnet_b0",
#     "vgg16",
    "resnet50",
#     "resnet18",
#     "vit_b_16" # support only image_size of (224, 224)
]

In [117]:
ISIC_df = pd.read_csv(CSV_PATH)
ISIC_df = ISIC_df[["isic_id", "patient_id", "target"]]
ISIC_df["img_path"] = ISIC_df["isic_id"].apply(lambda id: f"{os.path.join(IMG_DIR, id)}.jpg")
irrelevant_imgs_df = pd.read_csv(IRRELEVANT_IMGS_PATH)
ISIC_Ddf = ISIC_df[~ISIC_df["isic_id"].isin(irrelevant_imgs_df["img_name"])]
ISIC_df.drop(["isic_id"], axis=1, inplace=True)

/var/folders/tw/t4_6l2cn5hz3hqm6tjzgwk5c0000gn/T/ipykernel_45179/2637853055.py:1: DtypeWarning: Columns (51,52) have mixed types. Specify dtype option on import or set low_memory=False.
  ISIC_df = pd.read_csv(CSV_PATH)


,patient_id,target,img_path
0,IP_1235828,0,/Users/yuda/Desktop/data_bases/isic-2024-chall...
1,IP_8170065,0,/Users/yuda/Desktop/data_bases/isic-2024-chall...
2,IP_6724798,0,/Users/yuda/Desktop/data_bases/isic-2024-chall...
3,IP_4111386,0,/Users/yuda/Desktop/data_bases/isic-2024-chall...
4,IP_8313778,0,/Users/yuda/Desktop/data_bases/isic-2024-chall...
...,...,...,...
401054,IP_1140263,0,/Users/yuda/Desktop/data_bases/isic-2024-chall...
401055,IP_5678181,0,/Users/yuda/Desktop/data_bases/isic-2024-chall...
401056,IP_0076153,0,/Users/yuda/Desktop/data_bases/isic-2024-chall...
401057,IP_5231513,0,/Users/yuda/Desktop/data_bases/isic-2024-chall...


kfold

In [119]:
groups = ISIC_df['patient_id']
k_fold = GroupKFold(n_splits=N_SPLITS)
df_arr = []
for (train_idx, val_idx) in k_fold.split(ISIC_df, groups=groups):
    df_arr.append([train_idx, val_idx])


ValueError: k-fold cross-validation requires at least one train/test split by setting n_splits=2 or more, got n_splits=1.

# ISIC_Dataset class

In [ ]:

def get_dataloader(df):
    dataset = ISICDataset(train_df, transform, augmentation_transform)
    dataloader = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=True)
    return dataloader


def get_dataloader_with_undersampling(train_df, val_df, train_class_0_size=10000, val_class_0_size=10000):
    print(type(train_df))
    train_class_1_size, val_class_1_size = len(train_df[train_df['target'] == 1]), len(val_df[val_df['target'] == 1])
    train_df.sample(frac=1)
    # under sampling
    undersample = RandomUnderSampler(sampling_strategy={0: train_class_0_size, 1: train_class_1_size})
    train_df, _ = undersample.fit_resample(train_df, train_df["target"])
    train_df = train_df.reset_index(drop=True)

    undersample = RandomUnderSampler(sampling_strategy={0: val_class_0_size, 1: val_class_1_size})
    val_df, _ = undersample.fit_resample(val_df, val_df["target"])
    val_df = val_df.reset_index(drop=True).iloc[::-1].reset_index(drop=True)  # to make al the class_1 in the beginning

    class_1_train_df = train_df[train_df["target"] == 1]
    class_0_train_df = train_df[train_df["target"] == 0]

    train_ds = ISICDataset(train_df, transform=transform, augmentation_transform=augmentation_transform)
    val_ds = ISICDataset(val_df, transform=transform)

    sampler = BalancedBatchSampler(class_0_train_df, class_1_train_df, batch_size=BATCH_SIZE,
                                   class_1_amount_per_batch=CLASS_1_AMOUNT_PER_BATCH)

    train_dl = DataLoader(train_ds, batch_size=BATCH_SIZE, sampler=sampler, generator=device_gen)
    val_dl = DataLoader(val_ds, batch_size=BATCH_SIZE, shuffle=False, generator=device_gen)

    return train_dl, val_dl

In [ ]:
def add_random_noise(image, noise_level=20):
    """
    Adds random noise to an image.
    
    Parameters:
    - image (PIL.Image.Image): The input image to which noise will be added.
    - noise_level (int): The maximum absolute value of the noise to add to each pixel channel.
    
    Returns:
    - PIL.Image.Image: The image with added random noise.
    """
    image_array = np.array(image)
    noise = np.random.randint(-noise_level, noise_level + 1, image_array.shape)
    noisy_image_array = image_array + noise
    noisy_image = Image.fromarray(noisy_image_array.astype(np.uint8))
    return noisy_image



transform = v2.Compose([
    v2.Resize(IMG_SIZE),
    v2.ToDtype(torch.float32, scale=True),
    v2.ToTensor(),
    v2.Normalize(mean = [0.6298, 0.5126, 0.4097], std = [0.1386, 0.1308, 0.1202]),
])

def augmentation_transform():
    transform = [
        v2.RandomRotation(degrees=(-180, 180), expand=False),
        v2.RandomPerspective(distortion_scale=0.5, p=0.5, interpolation=3),
        transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.1),
        v2.RandomHorizontalFlip(0.5),
        v2.RandomVerticalFlip(0.5),
        v2.RandomResizedCrop(IMG_SIZE, scale=(0.8, 1.2)),
        v2.RandomGrayscale(p=0.1),
        v2.Lambda(lambda img: add_random_noise(img)),
        v2.Lambda(lambda img: img),
        v2.Lambda(lambda img: img),
        v2.Lambda(lambda img: img),
        
#         v2.Lambda(lambda img: add_gaussian_noise(img, mean=0, std=0.1)),  # Custom function to add Gaussian noise
#         v2.Lambda(lambda img: elastic_transform(img, alpha=34, sigma=4)),  # Custom function to apply elastic transformations]
    ]
    return random.choice(transform)

val_transform = v2.Compose([
    v2.RandomHorizontalFlip(0.23),
    v2.RandomVerticalFlip(0.23),
])

In [ ]:
class ISICDataset(Dataset):
    
    def __init__(self, df, transform=None, augmentation_transform=None):
        self.df = df
        self.transform = transform
        self.augmentation_transform = augmentation_transform
    
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, idx):
        img = Image.open(self.df.loc[idx, "img_path"]).convert("RGB")
        img = v2.Resize(IMG_SIZE)(img)
        img = v2.ToDtype(torch.float32, scale=True)(img)
        label = self.df.loc[idx, "target"]
        if self.augmentation_transform:    
            img = self.augmentation_transform()(img) if random.random() > 0.2 else img
        img = self.transform(img)
        # if random.random() > 0.98:
        #     plt.imshow(img.permute(1, 2, 0))
        #     plt.show()
        return img, label



In [ ]:
class BalancedBatchSampler(Sampler):

    def __init__(self, class_0_df, class_1_df, batch_size, class_1_amount_per_batch=10):
        self.class_0_df = class_0_df
        self.class_1_df = class_1_df

        self.batch_size = batch_size
        self.class_1_amount_per_batch = class_1_amount_per_batch
        self.num_batches = (len(self.class_0_df) + len(self.class_1_df)) // batch_size

    def __iter__(self):
        class_0_iter = iter(self.class_0_df.index)
        class_1_iter = iter(self.class_1_df.index)
        
        for _ in range(self.num_batches):
            batch = []
            try:
                for _ in range(self.class_1_amount_per_batch):
                    batch.append(next(class_1_iter))
            except:
                class_1_iter = iter(self.class_1_df.index)
                batch = []
                for _ in range(self.class_1_amount_per_batch):
                    batch.append(next(class_1_iter))
            try:
                for _ in range(self.batch_size - self.class_1_amount_per_batch):
                    batch.append((next(class_0_iter)))
            except:
                class_0_iter = iter(self.class_0_df.index)
                for _ in range(self.batch_size - self.class_1_amount_per_batch):
                    batch.append(next(class_0_iter))
            random.shuffle(batch)
            for idx in batch:
                yield idx

    def __len__(self):
        return self.num_batches


# model

In [ ]:
def load_model(model_name="resnet50", num_classes=2):

    if model_name == "mobilenet_v3_small":
        model = models.mobilenet_v3_small(weights=models.MobileNet_V3_Small_Weights.DEFAULT)
        model.classifier[3] = nn.Linear(model.classifier[3].in_features, num_classes)
    
    elif model_name == "efficientnet_v2_m":
        model = models.efficientnet_v2_m(weights=models.EfficientNet_V2_M_Weights.DEFAULT)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    
    elif model_name == "efficientnet_b0":
        model = models.efficientnet_b0(weights=models.EfficientNet_B0_Weights.DEFAULT)
        model.classifier[1] = nn.Linear(model.classifier[1].in_features, num_classes)
    
    elif model_name == "vgg16":
        model = models.vgg16(weights=models.VGG16_Weights.DEFAULT)
        model.classifier[6] = nn.Linear(model.classifier[6].in_features, num_classes)
    
    elif model_name == "resnet50":
        model = models.resnet50(weights=models.ResNet50_Weights.DEFAULT)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    
    elif model_name == "resnet18":
        model = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)
        model.fc = nn.Linear(model.fc.in_features, num_classes)
    
    elif model_name == "vit_b_16":
        model = models.vit_b_16(weights=models.ViT_B_16_Weights.DEFAULT)
        in_features = model.heads.head.in_features
        model.heads.head = nn.Linear(in_features, num_classes)
    else:
        raise ValueError(f"Model {model_name} is not supported yet")
        
#     print(model)
    model = model.to(device)
    
    return model


In [ ]:
k_models = []
for model_name in MODELS_NAMES:
    k_models.append(load_model(model_name=model_name, num_classes=NUM_CLASSES))

optimizer = AdamW([param for model in k_models for param in model.parameters()], lr=LR)


class_weights = compute_class_weight('balanced', classes=[0, 1], y=ISIC_df['target'])
weights = torch.tensor(class_weights, dtype=torch.float32)  # increase cause model will pay more in mistakes
criterion =  nn.CrossEntropyLoss(weight=weights) if NUM_CLASSES == 2 else nn.BCELoss()


scheduler = torch.optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0)

# training func

In [ ]:
def train(dataloader, k_models, optimizer, criterion):
    total_loss = 0
    sum_batches = 0
    for batch, (imgs, labels) in enumerate(dataloader):
        sum_batches += 1
        imgs, labels = imgs.to(device), labels.to(device)
        
        if NUM_CLASSES == 2:
            pred_labels = torch.zeros(imgs.size(0), 2, dtype=torch.float32, device=device)
            for model in k_models:
                model.train()
                pred_labels = pred_labels + model(imgs)
            pred_labels = pred_labels / len(k_models)
        else:
            raise Exception(f"define NUM_CLASSES")
        
        loss = criterion(pred_labels, labels)
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        
        # try:
        #     scheduler.step()
        # except:
        #     print('cosine warmup ERROR')
        #     
        total_loss += loss.item()
        lr_values.append(optimizer.param_groups[0]['lr'])
        if batch % 10 == 0:
            print(f"{batch + 1} from {len(dataloader) * BATCH_SIZE}, lr: {optimizer.param_groups[0]['lr']}")
    return total_loss / sum_batches


In [ ]:
def test(dataloader, model, criterion):
    model.eval()
    pred_arr = [0, 1] # initialize for cases where only class_0 or only class_1 
    labels_arr = [1, 0]
    total_loss = 0

    with torch.no_grad():
        for batch, (imgs, labels) in enumerate(dataloader):
            imgs, labels = imgs.to(device), labels.to(device)
            if NUM_CLASSES == 1:
                labels = labels.unsqueeze(1).float()
    
                pred_labels = model(imgs)
                pred_labels = torch.sigmoid(pred_labels)
                loss = criterion(pred_labels, labels)
                
                total_loss += loss.item()
                pred_arr = np.concatenate((pred_arr, pred_labels.cpu().flatten().detach().numpy()), axis=None)
                labels_arr = np.concatenate((labels_arr, labels.cpu().flatten().detach().numpy()), axis=None)
                
            elif NUM_CLASSES == 2:
                pred_labels = torch.zeros(imgs.size(0), 2, dtype=torch.float32, device=device)
                for model in k_models:
                    model.eval()
                    pred_labels = pred_labels + model(imgs)
                pred_labels = pred_labels / len(k_models)
                
                loss = criterion(pred_labels, labels)
                total_loss += loss.item()
                probabilities = nn.functional.softmax(pred_labels, dim=1)
                
                pred_arr = pred_arr + probabilities[:, 1].cpu().flatten().detach().numpy().tolist()
                labels_arr = labels_arr + labels.detach().flatten().cpu().numpy().tolist()
                
            else:
                raise Exception(f"define NUM_CLASSES")
            
            if batch % 5 == 0:
                print(f"{batch + 1} from {len(dataloader)}, loss: {loss}, auc: {roc_auc_80(pred_arr, labels_arr)}")
    return total_loss / len(dataloader), roc_auc_80(pred_arr, labels_arr)

## running (and initialize the data) 

In [ ]:
k_models = []
lr_values = []

final_arr = []
for fold, (train_df, val_df) in enumerate(k_fold):
    
    train_df = ISIC_df.loc[train_df, ['img_path', 'target']]
    val_df = ISIC_df.loc[val_df, ['img_path', 'target']]
    
    train_loss_arr = []
    test_loss_arr = []
    test_auc_arr = []
    gap = "------------------------------"
    for epoch in range(NUM_EPOCHS):
        train_dataloader, val_dataloader = get_dataloader_with_undersampling(train_df, val_df, 2000, 2000)
    
        print(f"{gap}\nepoch: {epoch + 1}\n{gap}")
    
        print("train:")
        train_loss_arr.append(train(train_dataloader, k_models[:fold] + k_models[fold + 1:], optimizer, criterion))
        print("test:")
        loss, auc_ = test(val_dataloader, k_models[:fold] + k_models[fold + 1:], criterion)
    
        test_loss_arr.append(loss)
        test_auc_arr.append(auc_)
        
        print(f"train_loss_arr: {train_loss_arr[-1]} ; test_loss_arr: {test_loss_arr[-1]} ; test_auc_arr: {test_auc_arr[-1]} ;")
        if test_auc_arr[-1] > 0.16:
            for i in range(len(k_models)):
                torch.save(k_models[i].state_dict(), f"model_{epoch + 1}_{str(test_loss_arr[-1])[2:5]}_{MODELS_NAMES[i]}_{NAMES_SAVED_MODELS}.pth")
            print(f"Saved PyTorch Models State to models_{epoch + 1}.pth")
        
    final_arr.append(pd.DataFrame({"train_loss_arr":train_loss_arr, "test_loss_arr":test_loss_arr, "test_auc_arr":test_auc_arr}))
    print("done!")

# get prediction with kfolds for adding into tabular

In [ ]:
for fold, (train_df, val_df) in enumerate(k_fold):
    
    train_df = ISIC_df.loc[train_df, ['img_path', 'target']]
    val_df = ISIC_df.loc[val_df, ['img_path', 'target']]
    
    model, optimizer, criterion = k_models[fold]
    
    test_loss_arr = []
    test_auc_arr = []
    gap = "------------------------------"
    for epoch in range(NUM_EPOCHS):
        train_dataloader, val_dataloader = get_dataloader_with_undersampling(train_df, val_df, 2000, 2000)
    
        print(f"{gap}\nepoch: {epoch + 1}\n{gap}")
        print("test:")
        loss, auc_ = test(val_dataloader, model, criterion)
    
        test_loss_arr.append(loss)
        test_auc_arr.append(auc_)
        # torch.save(model.state_dict(), f"model_{epoch + 1}.pth")
        # print(f"Saved PyTorch Model State to model_{epoch + 1}.pth")
        print(
            f"test_loss_arr: {test_loss_arr[-1]} ; test_auc_arr: {test_auc_arr[-1]} ;")
        
    k_models[fold] = (model, optimizer, criterion)
    pd.DataFrame({"test_loss_arr":test_loss_arr, "test_auc_arr":test_auc_arr})
    print("done!")

# need a small corrections. its temporarily